# Motivation

1) Calculate prompt perplexity
2) Calculate perplexity of ONLY the PREDICTED token conditioned on prompt
3) Calculate perplexity of ONLY the TARGET token conditions on prompt

Example:
Prompt = "My name is"
Predicted = "John"
Target = "Dennis"

In [1]:
import torch
import torch.nn.functional as F
from models import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

In [2]:
model = QuestionAnswerModel("google/flan-t5-small")
tokenizer = model.tokenizer

prompt = "My name is"
answer = " Dennis"

# NLL of only PREDICTED/TARGET token(s)

In [3]:
# NLL of entire sequence
inputs, outputs = model.calculate_perplexity(prompt, answer, mask_prompt=False)
logits = outputs.logits.reshape(-1, outputs.logits.shape[-1])
target_ids = inputs.reshape(-1)

nll_loss = F.cross_entropy(logits, target_ids, reduction="none")
nll_loss.mean(), outputs.loss

(tensor(2.0421), tensor(2.0421))

In [4]:
# NLL of model response
inputs, outputs = model.calculate_perplexity(prompt, answer, mask_prompt=True)
logits = outputs.logits.reshape(-1, outputs.logits.shape[-1])
target_ids = inputs.reshape(-1)

nll_loss = F.cross_entropy(logits, target_ids, reduction="none")
nll_loss[-1], outputs.loss

(tensor(7.8655), tensor(7.8655))

# Why is the NLL differ between subsequences?

In [5]:
# Perplexity of ONLY prompt given prompt + answer ("My name is Dennis")
inputs, outputs = model.calculate_perplexity(prompt, answer, mask_prompt=False)
logits = outputs.logits.reshape(-1, outputs.logits.shape[-1])
target_ids = inputs.reshape(-1)

nll_loss = F.cross_entropy(logits, target_ids, reduction="none")
nll_loss[:-2].mean(), tokenizer.decode(target_ids[:-2])

(tensor(1.8402), 'My name is')

In [6]:
# Perplexity of ONLY prompt given prompt ("My name is")
inputs, outputs = model.calculate_perplexity(prompt, "", mask_prompt=False)
logits = outputs.logits.reshape(-1, outputs.logits.shape[-1])
target_ids = inputs.reshape(-1)

nll_loss = F.cross_entropy(logits, target_ids, reduction="none")
nll_loss[:-1].mean(), tokenizer.decode(target_ids[:-1])

(tensor(1.2592), 'My name is')

1) Why is there a difference between perplexities between the subsequences
- Cross attention mechanism of transformers

2) How is the perplexity of the hypothesis paper calculated?
- Perplexity of ONLY prompt given prompt (latter case)

3) Do we include the model's reponse when calculating the prompt's perplexity? \n
- No